In [ ]:
 Import Required Libraries

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
Define Model Architecture (Same as Training)

In [ ]:
def create_soil_model():
    """Create the same EfficientNet-B0 model as used in training"""
    model = models.efficientnet_b0(pretrained=False)  # No need for pretrained weights
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)
    return model

# Create model instance
model = create_soil_model()
model = model.to(device)
model.eval()
print("✅ Model architecture created")


In [ ]:
 Define Image Preprocessing (Same as Training)

In [ ]:
# Image preprocessing (exactly same as your training)
image_size = 224
val_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

print("✅ Image transforms defined")


In [ ]:
Load Trained Model Weights (Optional)

In [ ]:
def load_trained_model(model, model_path=None):
    """Load your trained model weights"""
    if model_path and os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location=device))
        print(f"✅ Model loaded from {model_path}")
    else:
        print("⚠️ No model path provided. Using current model weights.")
    
    model.eval()
    return model

# Load model (update path if you have saved weights)
model = load_trained_model(model, model_path=None)


In [ ]:
Define Test Dataset Class

In [ ]:
class TestDataset(Dataset):
    """Test dataset class for inference"""
    def __init__(self, ids, img_dir, transform=None):
        self.ids = ids
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_name = self.ids.loc[idx, 'image_id']
        img_path = os.path.join(self.img_dir, img_name)
        
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, img_name
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            # Return dummy image if loading fails
            dummy_image = torch.zeros(3, image_size, image_size)
            return dummy_image, img_name

print("✅ Test dataset class defined")


In [ ]:
 Load Test Data

In [ ]:
# Load test image IDs
test_ids_path = "/kaggle/input/soil-classification-part-2/soil_competition-2025/test_ids.csv"
test_dir = "/kaggle/input/soil-classification-part-2/soil_competition-2025/test"

test_ids = pd.read_csv(test_ids_path)
print(f"✅ Loaded {len(test_ids)} test images")

# Create test dataset and dataloader
test_dataset = TestDataset(test_ids, test_dir, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"✅ Test dataset created with {len(test_dataset)} images")


In [ ]:
Make Predictions

In [ ]:
def make_predictions(model, test_loader):
    """Make predictions on test data"""
    model.eval()
    predictions = []
    
    print("🔮 Making predictions...")
    
    with torch.no_grad():
        for batch_idx, (images, image_names) in enumerate(test_loader):
            images = images.to(device)
            outputs = model(images)
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).int().cpu().numpy().flatten()
            
            for img_name, pred in zip(image_names, preds):
                predictions.append((img_name, pred))
            
            if (batch_idx + 1) % 10 == 0:
                print(f"Processed {(batch_idx + 1) * 32} images")
    
    print(f"✅ Predictions completed for {len(predictions)} images")
    return predictions

# Make predictions
predictions = make_predictions(model, test_loader)


In [ ]:
Create Submission File

In [ ]:
def create_submission(predictions, output_path="submission.csv"):
    """Create submission file"""
    # Convert to DataFrame
    submission = pd.DataFrame(predictions, columns=['image_id', 'label'])
    
    # Display statistics
    print("📊 Submission Statistics:")
    print(f"Total predictions: {len(submission)}")
    print(f"Soil predictions (1): {sum(submission['label'])}")
    print(f"Non-soil predictions (0): {len(submission) - sum(submission['label'])}")
    print(f"Soil percentage: {(sum(submission['label']) / len(submission)) * 100:.2f}%")
    
    # Save to CSV
    submission.to_csv(output_path, index=False)
    print(f"✅ {output_path} saved!")
    
    # Display sample
    print("\n📋 Sample predictions:")
    print(submission.head())
    
    return submission

# Create submission
submission_df = create_submission(predictions, "final_submission.csv")
